In [78]:
import pytesseract
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rotate
from deskew import determine_skew
import glob
import cv2
import numpy as np  
import os
from tqdm.notebook import tqdm 
import time
import pickle


#ignore UserWarning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [79]:
def deskew(train_or_test, more_or_less):
    temp = ""
    if more_or_less == "less":
        temp = "less30"
    else:
        temp = "moreeq30"

    
    input_list_names = glob.glob(f"wszystkie_zdj/{train_or_test}/*",)
    

    for photo_name in tqdm(input_list_names):
        name = photo_name.split("\\")[-1]
        name = name.split(".")[0]

        try:    
            image = io.imread(photo_name)
        except:
            print(f"Nie udało się otworzyć pliku {name}")
            continue
        angle = determine_skew(image)
        rotated = rotate(image, angle, resize=True) * 255

        if abs (angle) >= 30:
            io.imsave(f'deskewed/{train_or_test}/{temp}/{name}.png', image.astype(np.uint8))
        else:
            io.imsave(f'deskewed/{train_or_test}/{temp}/{name}.png', rotated.astype(np.uint8))

In [80]:
def increase_contrast(train_or_test, more_or_less):
    # increase contrast
    temp = ""
    if more_or_less == "less":
        temp = "less30"
    else:
        temp = "moreeq30"

    
    input_list_names = glob.glob(f"deskewed/{train_or_test}/{temp}/*",)
    for photo_name in tqdm(input_list_names):
        name = photo_name.split("\\")[-1]
        name = name.split(".")[0]
        original_image = io.imread(photo_name)
        image = Image.fromarray(original_image)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.25)
        image = np.array(image)
        io.imsave(f'contrast/{train_or_test}/{temp}/{name}.png', image.astype(np.uint8))


In [81]:
def sharpen(train_or_test, more_or_less):
    # increase contrast
    temp = ""
    if more_or_less == "less":
        temp = "less30"
    else:
        temp = "moreeq30"
    input_list_names = glob.glob(f"contrast/{train_or_test}/{temp}/*",)
    for photo_name in tqdm(input_list_names):
        name = photo_name.split("\\")[-1]
        name = name.split(".")[0]
        original_image = io.imread(photo_name)
        sharp_image = cv2.filter2D(original_image, -1, sharpen_filter)
        sharp_image = Image.fromarray(sharp_image)
        sharp_image.save(f'sharpened/{train_or_test}/{temp}/{name}.png')

In [82]:
def get_text(train_or_test, more_or_less):
    temp = ""
    if more_or_less == "less":
        temp = "less30"
    else:
        temp = "moreeq30"
    input_list_names = glob.glob(f"sharpened/{train_or_test}/{temp}/*",)
    custom_config = r'--oem 3 --psm 6 -l pol'
    photos_dict = {}
    for photo_name in tqdm(input_list_names):
        name = photo_name.split("\\")[-1]
        name = name.split(".")[0]
        image = io.imread(photo_name)


        # Perform OCR using pytesseract
        text = pytesseract.image_to_string(image, config=custom_config)
        # count number of \n
        count = text.count("\n")
        # replace \n with space
        text = text.replace("\n", " ")
        # replace \x0c with space
        text = text.replace("\x0c", " ")
    

        photos_dict[name] = (text, count)
    with open(f'photos_dict_{temp}_{train_or_test}.pickle', 'wb') as handle:
        pickle.dump(photos_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return photos_dict

In [83]:
def pipeline(train_or_test, more_or_less, deskew_ = True, increase_contrast_ = True, sharpen_ = True, get_text_ = True):
    temp = ""
    if more_or_less == "less":
        temp = "less30"
    else:
        temp = "moreeq30"

    if deskew_:
        deskew(train_or_test, more_or_less)
    
    if increase_contrast_:
        increase_contrast(train_or_test, more_or_less)
    
    if sharpen_:
        sharpen(train_or_test, more_or_less)
    
    if get_text_:
        get_text(train_or_test, more_or_less)
    # read pickle
    with open(f'photos_dict_{temp}_{train_or_test}.pickle', 'rb') as handle:
        photos_dict = pickle.load(handle)
    return photos_dict

In [84]:
pipeline("test", "more", deskew_=False)
pipeline("train", "more", deskew_=False)
pipeline("test", "less", deskew_=False)
pipeline("train", "less", deskew_=False)


  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/3456 [00:00<?, ?it/s]

  0%|          | 0/3456 [00:00<?, ?it/s]

  0%|          | 0/3456 [00:00<?, ?it/s]

  0%|          | 0/10742 [00:00<?, ?it/s]